## Install libraries

In [1]:
!pip install --upgrade gspread oauth2client
!pip install pinecone-client
!pip install --upgrade torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install sentence-transformers
!pip install streamlit
!pip install streamlit pyngrok
!pip install gradio pyngrok sentence-transformers pandas



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 1.1 MB/s eta 0:00:00
  Attempting uninstall: gspread
    Found existing installation: gspread 6.0.2
    Uninstalling gspread-6.0.2:
      Successfully uninstalled gspread-6.0.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.8/244.8 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 3.8 MB/s eta 0:00:00
Looking in indexes: https://download.pytorch.org/whl/cu118
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 838.3/838.3 MB 703.7 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 80.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 46.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.2/23.2 MB 73.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 875.6/875.6 kB 35.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 80.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 663.9/663.9 MB 1

## Import libraries

In [2]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from google.colab import files
from datetime import datetime
import pandas as pd
import numpy as np
import random
from sentence_transformers import SentenceTransformer, util
import os
import pinecone
from pinecone import Pinecone
import gradio as gr
from sklearn.metrics.pairwise import cosine_similarity


/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


## Authenticate and connect to Google Sheets

In [3]:

uploaded = files.upload() # Manually upload your service_account.json file here

# Replace 'your_service_account.json' with the name of the uploaded JSON key file
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
creds = ServiceAccountCredentials.from_json_keyfile_name('scrum-call-assistant-a8750ee31f37.json', scope)
client = gspread.authorize(creds)


# Function to create a new worksheet for each day
def create_daily_sheet():
    # Get today's date in YYYY-MM-DD format
    today_date = datetime.now().strftime("%Y-%m-%d")

    # Check if a worksheet with today's date already exists
    try:
        worksheet = spreadsheet.worksheet(today_date)
        print(f"Worksheet for {today_date} already exists.")
    except gspread.exceptions.WorksheetNotFound:
        # Create a new worksheet for today if it doesn't exist
        worksheet = spreadsheet.add_worksheet(title=today_date, rows="100", cols="5")
        headers = ["Project Name", "Project ID", "Employee Names", "Updates", "Blockers/Queries"]
        worksheet.append_row(headers)
        print(f"New worksheet created for {today_date}.")

    return worksheet

spreadsheet = client.create("Employee Updates Tracker")
spreadsheet.share('sudiksha-chindula@scrum-call-assistant.iam.gserviceaccount.com', perm_type='user', role='writer')  # Share with your email or anyone who needs access
spreadsheet.share('sudiksha.chindula7@gmail.com', perm_type='user', role='writer')

daily_sheet = create_daily_sheet()

worksheet = spreadsheet.get_worksheet(0)
worksheet.update_title("Daily Updates")



print(f"Spreadsheet created: {spreadsheet.url}")


Saving scrum-call-assistant-a8750ee31f37.json to scrum-call-assistant-a8750ee31f37.json
New worksheet created for 2024-11-10.
Spreadsheet created: https://docs.google.com/spreadsheets/d/1NBSlqiqf6jd157vUt7wBOQ2p8WFWIPIJb1P8ZRKhVBY


## Sample entries for the day

In [4]:
import random
from datetime import datetime, timedelta
import gspread
from oauth2client.service_account import ServiceAccountCredentials

# Authenticate with Google Sheets
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
creds = ServiceAccountCredentials.from_json_keyfile_name('scrum-call-assistant-a8750ee31f37.json', scope)
client = gspread.authorize(creds)

# Open the main spreadsheet and create or access today's worksheet
spreadsheet = client.open("Employee Updates Tracker")
today_date = datetime.now().strftime("%Y-%m-%d")

# Function to get or create the daily sheet
def get_or_create_daily_sheet():
    try:
        worksheet = spreadsheet.worksheet(today_date)
        print(f"Worksheet for {today_date} already exists.")
    except gspread.exceptions.WorksheetNotFound:
        worksheet = spreadsheet.add_worksheet(title=today_date, rows="100", cols="5")
        headers = ["Project Name", "Project ID", "Employee Names", "Updates", "Blockers/Queries"]
        worksheet.append_row(headers)
        print(f"New worksheet created for {today_date}.")
    return worksheet

# Access today's worksheet
worksheet = get_or_create_daily_sheet()

# Sample data generation for 'Employee Updates Tracker'
projects = ["Data Pipeline Enhancement", "Web App Redesign", "API Integration", "Machine Learning Model", "Backend Optimization"]
employees = ["Alice", "Bob", "Charlie", "Diana", "Evan"]
blockers = ["None", "Waiting for data access", "Dependency on API update", "Awaiting feedback", "Issue with deployment"]
updates = [
    "Completed initial setup.",
    "Working on API endpoints.",
    "Refactoring code for efficiency.",
    "Testing the latest model.",
    "Resolving deployment issues.",
    "Code review completed.",
    "Fixed bugs reported in QA.",
    "Integrating third-party API.",
    "Researching optimization techniques.",
    "Finalizing documentation."
]

# Generate and add 25 entries to today's worksheet
for i in range(10):
    project = random.choice(projects)
    employee = random.choice(employees)
    update = random.choice(updates)
    blocker = random.choice(blockers)
    project_id = f"PID-{random.randint(1000, 9999)}"

    # Prepare the row data
    row = [project, project_id, employee, update, blocker]

    # Append the row to the worksheet
    worksheet.append_row(row)
    print(f"Added entry: {row}")

print(f"Entries successfully added to the worksheet for {today_date}.")


Worksheet for 2024-11-10 already exists.
Added entry: ['API Integration', 'PID-4961', 'Diana', 'Fixed bugs reported in QA.', 'Waiting for data access']
Added entry: ['Web App Redesign', 'PID-3145', 'Diana', 'Code review completed.', 'None']
Added entry: ['Data Pipeline Enhancement', 'PID-2887', 'Evan', 'Integrating third-party API.', 'Awaiting feedback']
Added entry: ['Web App Redesign', 'PID-4601', 'Diana', 'Code review completed.', 'Issue with deployment']
Added entry: ['Data Pipeline Enhancement', 'PID-2100', 'Diana', 'Working on API endpoints.', 'None']
Added entry: ['API Integration', 'PID-8618', 'Evan', 'Code review completed.', 'Issue with deployment']
Added entry: ['Data Pipeline Enhancement', 'PID-6506', 'Diana', 'Finalizing documentation.', 'Awaiting feedback']
Added entry: ['API Integration', 'PID-5206', 'Bob', 'Completed initial setup.', 'Issue with deployment']
Added entry: ['Data Pipeline Enhancement', 'PID-9684', 'Diana', 'Researching optimization techniques.', 'Waiting 

## Script to fetch data

In [5]:
'''
COMPLETE CODE FOR A FILE ALREADY CREATED

import gspread
from oauth2client.service_account import ServiceAccountCredentials
from datetime import datetime
import pandas as pd

# Authenticate with Google Sheets
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
creds = ServiceAccountCredentials.from_json_keyfile_name('scrum-call-assistant-a8750ee31f37.json', scope)
client = gspread.authorize(creds)

# Open the main spreadsheet
spreadsheet = client.open("Employee Updates Tracker")
'''

# Function to extract data from today's worksheet
def extract_today_data():
    today_date = datetime.now().strftime("%Y-%m-%d")
    try:
        worksheet = spreadsheet.worksheet(today_date)
        # Get all records as dictionaries
        records = worksheet.get_all_records()
        print(f"Data for {today_date} extracted successfully.")
        # Convert to a pandas DataFrame for tabular format (optional)
        data_df = pd.DataFrame(records)
        return records, data_df
    except gspread.exceptions.WorksheetNotFound:
        print(f"No worksheet found for {today_date}.")
        return [], None



# Extract data for today in list and DataFrame format
today_data_list, today_data_df = extract_today_data()

# Print today's data in list format
print("Today's Data List:")
print(today_data_list)

# Print today's data in tabular format (if using pandas DataFrame)
if today_data_df is not None:
    print("\nToday's Data Table:")
    print(today_data_df)


Data for 2024-11-10 extracted successfully.
Today's Data List:
[{'Project Name': 'API Integration', 'Project ID': 'PID-4961', 'Employee Names': 'Diana', 'Updates': 'Fixed bugs reported in QA.', 'Blockers/Queries': 'Waiting for data access'}, {'Project Name': 'Web App Redesign', 'Project ID': 'PID-3145', 'Employee Names': 'Diana', 'Updates': 'Code review completed.', 'Blockers/Queries': 'None'}, {'Project Name': 'Data Pipeline Enhancement', 'Project ID': 'PID-2887', 'Employee Names': 'Evan', 'Updates': 'Integrating third-party API.', 'Blockers/Queries': 'Awaiting feedback'}, {'Project Name': 'Web App Redesign', 'Project ID': 'PID-4601', 'Employee Names': 'Diana', 'Updates': 'Code review completed.', 'Blockers/Queries': 'Issue with deployment'}, {'Project Name': 'Data Pipeline Enhancement', 'Project ID': 'PID-2100', 'Employee Names': 'Diana', 'Updates': 'Working on API endpoints.', 'Blockers/Queries': 'None'}, {'Project Name': 'API Integration', 'Project ID': 'PID-8618', 'Employee Names'

## Loading Embedding Moder and Connecting to a Vector Database

In [6]:



# Load SBERT model for embeddings
model = SentenceTransformer('all-MiniLM-L6-v2')

os.environ["PINECONE_API_KEY"] = "pcsk_51kFav_LVbf4tK5bnDvjEkWoFQnLq5iBagRstRhoVR2d3UjAkVty2uCxNCo7Csv9aXzZjP"
os.environ["PINECONE_ENVIRONMENT"] = "aped-4627-b74a"
os.environ["PINECONE_INDEX_NAME"] = "employee-updates-tracker"  # Ensure this line is run to set the index name

# Initialize Pinecone without using init()

index_name = os.getenv("PINECONE_INDEX_NAME", "employee-updates-tracker")  # Default to "text-embeddings" if not set
pc = Pinecone(api_key=os.environ.get("PINECONE_API_KEY"), environment=os.environ.get("PINECONE_ENVIRONMENT"))

index = pc.Index(index_name)

# Access the index
index = pc.Index(index_name)

# Chunk data into manageable pieces if necessary
def chunk_text(text, max_chunk_size=100):
    return [text[i:i + max_chunk_size] for i in range(0, len(text), max_chunk_size)]

# Process and embed daily DataFrame
def embed_daily_data(data_df):
    index_vectors = []
    for _, row in data_df.iterrows():
        combined_text = f"Project: {row['Project Name']} | Update: {row['Updates']} | Blockers: {row['Blockers/Queries']}"
        chunks = chunk_text(combined_text, max_chunk_size=100)
        for chunk in chunks:
            vector = model.encode(chunk).tolist()
            index_vectors.append((f"{row['Project ID']}_{chunk}", vector))

    # Insert vectors into Pinecone index
    index.upsert(vectors=index_vectors)
    print(f"{len(index_vectors)} vectors upserted to Pinecone index.")

# Define the query_employee_updates function
def query_employee_updates(query, top_k=10):
    # Embed the query
    query_chunks = chunk_text(query, max_chunk_size=100)
    query_embeddings = [model.encode(chunk) for chunk in query_chunks]

    results = []
    for embedding in query_embeddings:
        pinecone_results = index.query(vector=embedding.tolist(), top_k=top_k, include_metadata=True)
        results.extend(pinecone_results['matches'])

    # Retrieve matching rows from the DataFrame based on Project ID
    relevant_rows = []
    for match in results:
        print("Match found:", match)  # Debug to see each match
        project_id = match['id'].split("_")[0]
        row = today_data_df[today_data_df['Project ID'] == project_id]
        if not row.empty:
            relevant_rows.append(row)

    return pd.concat(relevant_rows).drop_duplicates() if relevant_rows else pd.DataFrame()

# Check today's data in DataFrame
#print("Today's Data DataFrame:")
#print(today_data_df)

# Embed and index data
embed_daily_data(today_data_df)

# Example Query
query_text = "Issue with deployment"
print("\nRunning query:", query_text)

# Execute query
relevant_data = query_employee_updates(query_text)

# Display results
if not relevant_data.empty:
    print("\nRelevant Data:")
    print(relevant_data)
else:
    print("No relevant data found.")




/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

13 vectors upserted to Pinecone index.

Running query: Issue with deployment
Match found: {'id': 'PID-3328_deployment', 'score': 0.817429483, 'values': []}
Match found: {'id': 'PID-9945_Project: Backend Optimization | Update: Code review '
       'completed. | Blockers: Issue with deployment',
 'score': 0.532743156,
 'values': []}
Match found: {'id': 'PID-4772_Project: Backend Optimization | Update: Code review '
       'completed. | Blockers: Issue with deployment',
 'score': 0.532743156,
 'values': []}
Match found: {'id': 'PID-9142_Project: Backend Optimization | Update: Code review '
       'completed. | Blockers: Issue with deployment',
 'score': 0.532743156,
 'values': []}
Match found: {'id': 'PID-1874_Project: API Integration | Update: Resolving deployment '
       'issues. | Blockers: Issue with deployment',
 'score': 0.522117376,
 'values': []}
Match found: {'id': 'PID-1659_Project: Machine Learning Model | Update: Resolving '
       'deployment issues. | Blockers: Issue with d

## Integrating LLM and Streamlit UI

In [7]:
from sentence_transformers import SentenceTransformer
import numpy as np

# Load the SentenceTransformer model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Check if embeddings already exist in the DataFrame; otherwise, compute them
if 'embedding' not in today_data_df.columns:
    today_data_df['embedding'] = today_data_df['Updates'].apply(lambda x: model.encode(x))
else:
    print("Embeddings already exist in the DataFrame.")


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [8]:
from sklearn.metrics.pairwise import cosine_similarity

# Function to embed a query and find relevant data
def get_relevant_data(query_text):
    # Embed the query using the loaded model
    query_embedding = model.encode(query_text).reshape(1, -1)

    # Calculate cosine similarity between query and each row embedding
    df_embeddings = np.vstack(today_data_df['embedding'].values)
    similarities = cosine_similarity(query_embedding, df_embeddings).flatten()

    # Get top result(s) based on similarity
    top_indices = similarities.argsort()[-3:][::-1]  # Retrieve top 3 results
    relevant_rows = today_data_df.iloc[top_indices]
    return relevant_rows


last

In [9]:
import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import gradio as gr
'''
# Define sample data
data = {
    "Project Name": [
        "API Integration", "Machine Learning Model", "Web App Redesign",
        "Backend Optimization", "Data Pipeline Enhancement"
    ],
    "Project ID": [
        "PID-4741", "PID-7014", "PID-6011", "PID-3843", "PID-3015"
    ],
    "Employee Names": [
        "Alice", "Bob", "Evan", "Diana", "Charlie"
    ],
    "Updates": [
        "Resolving deployment issues.", "Researching optimization techniques.",
        "Fixed bugs reported in QA.", "Completed initial setup.",
        "Integrating third-party API."
    ],
    "Blockers/Queries": [
        "Dependency on API update", "None", "Awaiting feedback",
        "Dependency on API update", "None"
    ]
}

# Create the DataFrame
today_data_df = pd.DataFrame(data)
'''
# Load the SentenceTransformer model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Compute embeddings for each row in the DataFrame (only if not already present)
if 'embedding' not in today_data_df.columns:
    today_data_df['embedding'] = today_data_df['Updates'].apply(lambda x: model.encode(x))
else:
    print("Embeddings already exist in the DataFrame.")

# Function to embed a query and find relevant data
def get_relevant_data(query_text):
    # Embed the query using the loaded model
    query_embedding = model.encode(query_text).reshape(1, -1)

    # Calculate cosine similarity between query and each row embedding
    today_data_embeddings = np.vstack(today_data_df['embedding'].values)
    similarities = cosine_similarity(query_embedding, today_data_embeddings).flatten()

    # Get top result(s) based on similarity
    top_indices = similarities.argsort()[-3:][::-1]  # Retrieve top 3 results
    relevant_rows = today_data_df.iloc[top_indices]
    return relevant_rows

# Chatbot response function

def chatbot_response(history, user_input):
    # Get relevant data from the DataFrame based on the embedded query
    relevant_data = get_relevant_data(user_input)

    if relevant_data.empty:
        bot_reply = "No relevant data found."
    else:
        # Format the response based on relevant data
        bot_reply = ""
        for _, row in relevant_data.iterrows():
            bot_reply += f"**Project Name**: {row['Project Name']}\n"
            bot_reply += f"**Project ID**: {row['Project ID']}\n"
            bot_reply += f"**Employee Name**: {row['Employee Names']}\n"
            bot_reply += f"**Update**: {row['Updates']}\n"
            bot_reply += f"**Blockers/Queries**: {row['Blockers/Queries']}\n\n"
        bot_reply = bot_reply.strip()

    # Append the user's input and bot's reply to the chat history
    history.append(["user", user_input])
    history.append(["bot", bot_reply])

    return history, history

# Define the Gradio interface for the chatbot
with gr.Blocks() as demo:
    gr.Markdown("<h1 style='text-align: center;'>Employee Updates Chatbot</h1>")
    chatbot = gr.Chatbot()
    msg = gr.Textbox(placeholder="Type a message...", label="Enter your query:")
    submit = gr.Button("Send")

    # Set the function with inputs and outputs for the chatbot
    submit.click(fn=chatbot_response, inputs=[chatbot, msg], outputs=[chatbot, chatbot])

# Launch the Gradio app with sharing enabled
demo.launch(share=True)


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Embeddings already exist in the DataFrame.


/usr/local/lib/python3.10/dist-packages/gradio/components/chatbot.py:223: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  warnings.warn(


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://0f7ec347d7344ae6a2.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
